<a href="https://colab.research.google.com/github/RybakovAlex/Python-for-Analythics/blob/main/%D0%91%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82_%22GB_Homework_6_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Урок 6. Линейная алгебра с библиотекой Numpy

Решить кейс:
Руководство решает внедрить фичу “С этим товаром покупают” в Интернет-магазине. Вам предлагается протестировать фичу на одном из товаров. Для тестирования фичи вам исходя из истории покупок в интернет-магазине нужно определить ТОП-10 наиболее близких товаров к исходному.
Используйте датасет с практики текущего урока.
Создайте матрицу item-customer (по срокам - товары, по столбцам - покупатели)
Проведите оценку мер близости товаров, получив матрицу item_item_sim_matrix со значениями косинусов между векторами товаров.
Отберите ТОП-10 похожих товаров по StockCode.
Выведите список ТОП-10 похожих товаров с названиями (Description) на экран.
Исходный товар - StockCode: 23166 Description: MEDIUM CERAMIC TOP STORAGE JAR

In [97]:
# Импортирую нужные библиотеки
import pandas as pd
import numpy as np

1.  Используйте датасет с практики текущего урока.
Источник и описание полей датасета: http://archive.ics.uci.edu/ml/datasets/online+retail

In [98]:
# Загружаю и читаю датасет
df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [99]:
# Смотрю размеры датасета
df.shape

(541909, 8)

2. Создайте матрицу item-customer (по срокам - товары, по столбцам - покупатели)

In [100]:
# Чищу датасет: удаляю записи с нулевыми покупками
df = df.loc[df['Quantity'] > 0]

In [101]:
# Чищу датасет: удаляю записи с пропусками в customer_id
df = df.dropna(subset=['CustomerID'])

In [102]:
# Агрегирую данные в сводную таблицу "покупатель - товар"
item_customer = df.pivot_table(index='StockCode', columns='CustomerID', values='Quantity', aggfunc='sum')
item_customer.head()

CustomerID,12346.0,12347.0,12348.0,12349.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,...,18273.0,18274.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
# Смотрю на размеры таблицы
item_customer.shape

(3665, 4339)

In [104]:
# Сравниваю кол-во строк с кол-вом уникальных покупателей в исходном датафрейме
df['CustomerID'].nunique()

4339

In [105]:
# Сравниваю кол-во столбцов с кол-вом уникальных товаров в исходном датафрейме
df['StockCode'].nunique()

3665

In [106]:
# Заменяю кол-во покупок на событие: 1- факт покупки, 0 - отсутствие покупки
item_customer = item_customer.applymap(lambda x: 1 if x > 0 else 0)
item_customer.head()

CustomerID,12346.0,12347.0,12348.0,12349.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,...,18273.0,18274.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10080,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10120,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10125,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10133,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


3. Проведите оценку мер близости товаров, получив матрицу item_item_sim_matrix со значениями косинусов между векторами товаров.

In [107]:
# Преобразую исходную матрицу "покупатель - товар" в массив numpy
m = item_customer.values
m

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [108]:
# Агрегирую данные в сводную таблицу "покупатель - товар"
item_customer = df.pivot_table(index='StockCode', columns='CustomerID', values='Quantity', aggfunc='sum')
item_customer.head()

CustomerID,12346.0,12347.0,12348.0,12349.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,...,18273.0,18274.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
# Импортирую библиотеку для создания разреженной матрицы
from scipy.sparse import csr_matrix

In [110]:
# Задаю функцию расчета косинуса между векторами-строками покупателей
def cos_sim(array):
    csr_array = csr_matrix(array)
# преобразую
    numerator = np.dot(csr_array,csr_array.T).toarray()
    denomenator = np.linalg.norm(array, axis = 1, keepdims = True)*np.linalg.norm(array.T, axis = 0, keepdims = True)
    return numerator / denomenator

In [111]:
# Вычисляю косинусную меру сходства между векторами-строками покупателей
item_item_sim_matrix = cos_sim(m)
item_item_sim_matrix

array([[1.        , 0.        , 0.09486833, ..., 0.06691496, 0.        ,
        0.07821652],
       [0.        , 1.        , 0.        , ..., 0.01618174, 0.        ,
        0.        ],
       [0.09486833, 0.        , 1.        , ..., 0.07053456, 0.        ,
        0.01099299],
       ...,
       [0.06691496, 0.01618174, 0.07053456, ..., 1.        , 0.        ,
        0.0775386 ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.07821652, 0.        , 0.01099299, ..., 0.0775386 , 0.        ,
        1.        ]])

In [112]:
 # Добавляю название столбцов и индексов в матрицу мер близости между векторами-строками покупателей
item_item_sim_matrix = pd.DataFrame(item_item_sim_matrix)
item_item_sim_matrix.columns = item_customer.index
item_item_sim_matrix['CustomerID'] = item_customer.index
item_item_sim_matrix = item_item_sim_matrix.set_index('CustomerID')
item_item_sim_matrix.head()

StockCode,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
10002,1.000000,0.000000,0.094868,0.090351,0.062932,0.098907,0.095346,0.047673,0.075593,0.090815,...,0.0,0.0,0.0,0.0,0.0,0.029361,0.0,0.066915,0.000000,0.078217
10080,0.000000,1.000000,0.000000,0.032774,0.045655,0.047836,0.000000,0.000000,0.082261,0.049413,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.016182,0.000000,0.000000
10120,0.094868,0.000000,1.000000,0.057143,0.059702,0.041703,0.060302,0.060302,0.095618,0.028718,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.070535,0.000000,0.010993
10125,0.090351,0.032774,0.057143,1.000000,0.042644,0.044682,0.043073,0.000000,0.051224,0.030770,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.070535,0.000000,0.070669
10133,0.062932,0.045655,0.059702,0.042644,1.000000,0.280097,0.045002,0.060003,0.071358,0.057152,...,0.0,0.0,0.0,0.0,0.0,0.036955,0.0,0.070185,0.049752,0.021877


4. Отберите ТОП-10 похожих товаров по StockCode.

In [115]:
# На примере эталонного товара нахожу те товары, которые близки к нему по косинусной мере
t1 = item_item_sim_matrix.loc[23166].sort_values(ascending=False)
t1.head(10)

StockCode
23166    1.000000
23165    0.671897
23167    0.659551
22993    0.274724
23307    0.262526
22722    0.249053
22720    0.249002
22666    0.247110
23243    0.246397
22961    0.242447
Name: 23166, dtype: float64

In [132]:
# Определяю перечень уникальных покупателей нашего эталонного товара
A = item_item_sim_matrix.loc[23166]
customer_bought_A = A[A>0].index.tolist()
customer_bought_A
A.head(10)

StockCode
10002    0.040379
10080    0.019529
10120    0.017025
10125    0.024322
10133    0.118584
10135    0.079875
11001    0.051333
15030    0.051333
15034    0.030523
15036    0.085563
Name: 23166, dtype: float64

In [133]:
# Определяю перечень уникальных товаров наиболее близкого к нашему
B = item_item_sim_matrix.loc[23165]
customer_bought_B = B[B>0].index.tolist()
customer_bought_B
B.head(10)

StockCode
10002    0.024470
10080    0.053258
10120    0.030953
10125    0.033164
10133    0.115497
10135    0.080677
11001    0.058329
15030    0.046663
15034    0.055494
15036    0.088892
Name: 23165, dtype: float64

In [143]:
# Выполняю пересечение множеств
customer_to_recommend_to_A = set(customer_bought_B) - set(customer_bought_A)
customer_to_recommend_to_A


{'16244B',
 '17090D',
 '17091A',
 20756,
 20760,
 20847,
 21030,
 21040,
 21087,
 21161,
 21277,
 21278,
 21316,
 21319,
 21320,
 21344,
 21547,
 21651,
 21694,
 21741,
 21758,
 21814,
 21847,
 21850,
 21851,
 21943,
 22042,
 22259,
 22264,
 22461,
 23465,
 '35607B',
 '35915C',
 35972,
 '47013C',
 71038,
 '77101A',
 78033,
 '79026B',
 '79144C',
 84358,
 84360,
 84499,
 '84562A',
 '84569B',
 84576,
 '84597B',
 84632,
 '84663A',
 84665,
 84666,
 84725,
 84773,
 '84796A',
 '84798A',
 '84800M',
 '84801B',
 '84802B',
 '84803B',
 '84805A',
 '84809A',
 '84876D',
 84881,
 84921,
 '84927A',
 '85028L',
 85030,
 '85049B',
 '85136B',
 '85230E',
 '85230F',
 '90016A',
 '90019A',
 '90036C',
 90050,
 '90081A',
 90099,
 90104,
 90116,
 '90124C',
 90144,
 '90160D',
 '90164A',
 '90197B',
 '90209B',
 '90210D',
 'BANK CHARGES'}

5. Выведите список ТОП-10 похожих товаров с названиями (Description) на экран.

In [147]:
# Выводим перечень ТОП-10 товаров эталонного покупателя
stock = df.loc[df['StockCode'].isin(customer_bought_A), ['StockCode', 'Description']].drop_duplicates().set_index('StockCode')
stock.head (10)

,Description
StockCode,
85123A,WHITE HANGING HEART T-LIGHT HOLDER
71053,WHITE METAL LANTERN
84406B,CREAM CUPID HEARTS COAT HANGER
84029G,KNITTED UNION FLAG HOT WATER BOTTLE
84029E,RED WOOLLY HOTTIE WHITE HEART.
22752,SET 7 BABUSHKA NESTING BOXES
21730,GLASS STAR FROSTED T-LIGHT HOLDER
22633,HAND WARMER UNION JACK
22632,HAND WARMER RED POLKA DOT


In [149]:
# Выводим перечень товаров, рекомендованных для нашего эталонного покупателя
df.loc[df['StockCode'].isin(customer_to_recommend_to_A), ['StockCode', 'Description']].drop_duplicates().set_index('StockCode')

,Description
StockCode,
17091A,LAVENDER INCENSE IN TIN
21651,HANGING GLASS ETCHED TEALIGHT
21814,HEART T-LIGHT HOLDER
84569B,PACK 3 FIRE ENGINE/CAR PATCHES
21694,SMALL REGAL SILVER CANDLEPOT
...,...
90050,CRACKED GLAZE NECKLACE IVORY
90116,FRUIT SALAD BAG CHARM
21344,MOROCCAN BEATEN METAL DISH
